<a href="https://colab.research.google.com/github/tourfade/uqtr-selmet/blob/main/DConvcomplexRegressionSelmet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os, io , skimage
import pandas as pd
from scipy.io import arff
import tensorflow as tf
from tensorflow.keras import activations, optimizers
from tqdm.keras import TqdmCallback
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Dropout, Flatten, AveragePooling1D, MaxPooling1D, GaussianNoise, Reshape, BatchNormalization,GlobalAveragePooling1D 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import urllib.request
import matplotlib.pyplot as plt
from google.colab import files
from skimage import data
from skimage.transform import rescale, resize, downscale_local_mean

In [ ]:
opt = optimizers.Adam(learning_rate=0.001,  epsilon=0.0001)
label = "omdcell"
p = ""
batch_size = 128
epochs = 250
percent_test = 0.15

#opt = optimizers.SGD( learning_rate=0.005, momentum=0.01, nesterov=True, name="SGD")
#opt = optimizers.RMSprop(learning_rate=0.0001)

In [ ]:
 
 
def build_dataset(df):
    Y = df[df.columns[-1]] 
    X = df[df.columns[0:-1]]
    X = X.values.reshape(X.shape[0], X.shape[1], 1)
    Y = Y.values.reshape(Y.shape[0], 1, 1)
    return (X, Y)

In [ ]:
# Train function
#def root_mean_squared_error(y_true, y_pred):
 #       return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def root_mean_squared_error(y,x):
        return (tf.sqrt(tf.reduce_mean(tf.square(x - y),axis = [0,1])))

def build_classifier(x_train, y_train, name):
    
    model = get_model()
    checkpoint_path = "train/"+name+".ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    model.compile(loss= tf.keras.losses.MSE,
                  optimizer = opt, 
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    # Create checkpoint callback
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                     save_weights_only = True,
                                                     verbose=0, 
                                                     callbacks=[TqdmCallback(verbose=2)])
    
    x_cal, x_val, y_cal, y_val = train_test_split(x_train, y_train, test_size=percent_test, shuffle= True)

    
    result = model.fit(x_cal, y_cal, 
                        validation_data=(x_val, y_val),
                        callbacks = [cp_callback], 
                        #steps_per_epoch = 406,
                        batch_size = batch_size,
                        shuffle = True,
                        epochs = epochs )
    model.summary()
    # summarize history for accuracy
    plt.figure( )
    plt.subplots(figsize = (10, 5))
    plt.title('accuracies-loss')
    rmqe = result.history['root_mean_squared_error']
    plt.plot(rmqe, color = 'g', label='train')
    val_rmqe = result.history['val_root_mean_squared_error']
    plt.plot(val_rmqe, color = 'r', label='val')
    norm_loss = result.history['loss']/np.max(result.history['loss'])
    plt.plot(norm_loss, color = 'b', label='loss')
    plt.legend()
    plt.show()
    return checkpoint_path

In [ ]:
def test_classifier(x_test, y_test, path, name):
    model = get_model()
    model.load_weights(path).expect_partial()
    model.compile(loss= tf.keras.losses.MSE,
                  optimizer=opt,
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
  
    
    print('(loss, rmse)', model.evaluate(x_test, y_test))
    return model
    

In [ ]:
def get_model():
    model = Sequential()
    model.add(Conv1D(filters=10, kernel_size=25,  activation=activations.relu, input_shape=(700, 1)))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=40, kernel_size=25, activation=activations.relu))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=60, kernel_size=25, activation=activations.relu))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=80, kernel_size=25, activation=activations.relu))
    model.add(AveragePooling1D())
    model.add(BatchNormalization())
    model.add(Conv1D(filters=100, kernel_size=25, activation=activations.relu))
    model.add(AveragePooling1D())
    model.add(BatchNormalization())

    model.add(Conv1D(filters=40, kernel_size=5, activation=activations.relu))
    model.add(MaxPooling1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=5, kernel_size=1, activation=activations.relu))

    model.add(GlobalAveragePooling1D())
    #model.add(Flatten())
   
    model.add(Dense(1, activation=activations.linear))
    return model

In [ ]:
#@title Titre par défaut
train_df = pd.read_csv("/content/drive/MyDrive/cirad/selmet/"+label+"/"+label+"_train_data"+p+".csv", sep=',')

x_train, y_train = build_dataset(train_df)
mean = x_train.mean();
std = x_train.std()
x_train = (x_train - mean) /std


In [ ]:
path =  build_classifier(x_train, y_train,  'model1') 


0epoch [00:00, ?epoch/s]

Epoch 1/250
45/45 [==============================] - 189s 4s/step - loss: 2675.3918 - root_mean_squared_error: 51.7242 - val_loss: 2541.4929 - val_root_mean_squared_error: 50.4132
Epoch 2/250
45/45 [==============================] - 186s 4s/step - loss: 2460.8904 - root_mean_squared_error: 49.6074 - val_loss: 2392.6418 - val_root_mean_squared_error: 48.9146
Epoch 3/250
45/45 [==============================] - 185s 4s/step - loss: 2214.7129 - root_mean_squared_error: 47.0607 - val_loss: 2104.9656 - val_root_mean_squared_error: 45.8799
Epoch 4/250
45/45 [==============================] - 185s 4s/step - loss: 1911.5598 - root_mean_squared_error: 43.7214 - val_loss: 1754.1624 - val_root_mean_squared_error: 41.8827
Epoch 5/250
45/45 [==============================] - 186s 4s/step - loss: 1565.6819 - root_mean_squared_error: 39.5687 - val_loss: 1377.4349 - val_root_mean_squared_error: 37.1138
Epoch 6/250
45/45 [==============================] - 181s 4s/step - loss: 1203.1073 - root_mean_squa

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/cirad/selmet/"+label+"/"+label+"_test_data"+p+".csv", sep=',')

x_test, y_test = build_dataset(test_df)
x_test =  (x_test - mean) /std


model = test_classifier(x_test, y_test, path, 'model1')

y_predicted =  model.predict(x_test) 
y_g_truth =    y_test

plt.figure( )
plt.subplots(figsize = (10, 5))
plt.title('plot')
 
x = list(range(0,y_test.shape[0]))
y_predicted = y_predicted.reshape(1,-1)[0]
plt.scatter(y_g_truth, y_predicted.transpose() ,  s=10, c ='r', label='pred')
#plt.scatter(x, y_g_truth ,  s=10, c  = 'r', label='truth')
plt.legend()
plt.show()


 